Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your collaborators below:

In [ ]:
COLLABORATORS = ""

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<h1 style="margin-top:0">Maximum-likelihood estimation</h1>
<br/>
<b>Maximum-likelihood estimation (MLE)</b> is a method of estimating the parameters of a statistical model given some data.

For example, we might be interested in the heights of students at Berkeley. Since we can't measure the heights of the entire population, we will need to use a sample. Assuming that the heights come from a normal distribution with some unknown mean and variance, the mean and variance can be estimated with MLE while only knowing the heights of some sample of the overall population. MLE accomplishes this by finding values for the paramters (mean and variance in this case) that <em>make the observed results the most probable given the model</em>.

Following previous examples in class, let's focus on a particular model that describes the behavior of a coin. A fair coin would have 0.5 probability of landing on heads, and a 0.5 probability of landing on tails. However, a coin can also be biased. We can capture this bias by finding the probability that each side of the coin will appear after being flipped. For example, we might have a 0.3 probability of gettings heads, and 0.7 for tails. This type of two-outcome distribution is called a **Bernoulli distribution**. This distribution only has one parameter $\theta$ ("theta"), since knowing the probability of heads will also tell us the probability of tails ($1-\theta$). So, our task will be to find the MLE of $\theta$ given some data.

<b>Calculating the Likelihood for a Series of Coin Flips</b>
<br>
Before we find the maximum likelihood estimate of $\theta$, we first need to know how to calculate how likely our data is given some arbitrary $\theta$. The likelihood of our data is simply the product of the probabilities of each datapoint (the joint probability). Recall that using our Bernoulli distribution, we will get heads with probability $\theta$, and tails with probability $1-\theta$. Mathematically, this corresponds to

$$P(d|\theta) = \theta^{N_H} \times (1-\theta)^{N_T} $$

where $d$ is the observed sequence of coin flips, $N_H$ is the number of Heads in the sequence, and $N_T$ is the number of tails in the sequence.

For example, if we set $\theta = 0.2$, the likelihood $P(d|\theta)$ of the sequence `T,H,T,T` will be $0.2 \times (1-0.2)^3 = 0.1024$.

### Coding Practice

Now, let's play around with come code for practice. First, let's create some datasets to play with. For work in Python, we'll denote `H`'s with a $1$ and `T`'s with a $0$.

In [ ]:
# slightly more heads than tails
data_1 = np.array([1,1,0,1,0,0,1,1,1,0])

# way more heads than tails
data_2 = np.array([1,1,0,1,1,1,0,1,1,1])

Second, we can define a function that returns the likelihood for a sequence of data. Note that this function is simply implementing the equation we wrote down earlier for $P(d|\theta)$

In [ ]:
def likelihood(data,theta):
    num_heads = np.sum(data)
    num_tails = np.sum(1-data)
    return (theta**num_heads) * ((1-theta)**num_tails)

We can evaluate this function on both datasets assuming $\theta = 0.6$ (i.e., that our coin was $60\%$ likely to produce heads on any given flip).

In [ ]:
print('P(data_1 | theta=0.6): {}'.format(likelihood(data_1,0.6)))
print('P(data_2 | theta=0.6): {}'.format(likelihood(data_2,0.6)))

Next, we can visualize the likelihood of the data as function many different possible values of theta between 0 and 1.

In [ ]:
# create 100 equally spaced values between 0 and 1
thetas = np.linspace(0,1,100)

In [ ]:
# take a quick look
thetas

In [ ]:
# compute the likelihood for each value of theta
likelihoods_1 = [likelihood(data_1, theta) for theta in thetas] # dataset 1
likelihoods_2 = [likelihood(data_2, theta) for theta in thetas] # dataset 2

In [ ]:
# plot likelihood as a function of theta for both datasets
plt.figure(figsize=(8,4))
plt.plot(thetas,likelihoods_1, c='blue', label='dataset 1')
plt.plot(thetas,likelihoods_2, c='green', label='dataset 2')
plt.title("Likelihood as a function of theta")
plt.ylabel("P(d|theta)")
plt.xlabel("theta")
plt.legend(loc='upper left')
plt.show()

The **maximum likelihood estimate (MLE)** of $\theta$ is the value that corresponds to the maximum likelihood of the data. Let's find this value by searching through the likelihoods we generated for the first dataset.

In [ ]:
# find the maximum likelihood value for each curve 
max_likelihood_1 = np.max(likelihoods_1)
max_likelihood_2 = np.max(likelihoods_2)

# find the values of thetas that corresponds to the maximum likelihood value
# for each curve
mle_1 = thetas[np.where(likelihoods_1 == max_likelihood_1)][0]
mle_2 = thetas[np.where(likelihoods_2 == max_likelihood_2)][0]

print('Maximum Likelihood Estimate for Theta under Dataset 1: {}'.format(mle_1))
print('Maximum Likelihood Estimate for Theta under Dataset 2: {}'.format(mle_2))

Let's visualize these.

In [ ]:
plt.figure(figsize=(8,4))

plt.plot(thetas,likelihoods_1,c='blue')
max_likelihood_1 = np.max(likelihoods_1)
mle_1 = thetas[np.where(likelihoods_1==max_likelihood_1)][0]
plt.plot(mle_1, max_likelihood_1,c='blue',marker='o')
plt.axvline(x=mle_1,c='blue')

plt.plot(thetas,likelihoods_2,c='green')
max_likelihood_2 = np.max(likelihoods_2)
mle_2 = thetas[np.where(likelihoods_2==max_likelihood_2)][0]
plt.plot(mle_2, max_likelihood_2,c='green',marker='o')
plt.axvline(x=mle_2,c='green')

plt.title("Likelihood as a function of theta")
plt.ylabel("Likelihood")
plt.xlabel("theta")
plt.show()

From the 100 values of $\theta$ that we evaluated earlier, we were able to find a single value that make the data most likely. However, we were only able to search 100 values for the best one, while there are actually infinitely many possible values that $\theta$ could take between 0 and 1. This means that the value we found may be a little off, which isn't good. Luckily, with a bit of math, it can be shown that $MLE(\theta) = \frac{N_H}{N_H + N_T}$. Let's try this now.

In [ ]:
# function to calculate MLE of theta
def calc_mle(data):  
    num_heads = np.sum(data)
    num_tails = np.sum(1-data)

    return num_heads / float((num_heads+num_tails))

In [ ]:
print('Analytical MLE for Dataset 1: {}'.format(calc_mle(data_1)))
print('Analytical MLE for Dataset 2: {}'.format(calc_mle(data_2)))

---

Before turning this problem in remember to do the following steps:

1. **Restart the kernel** (Kernel$\rightarrow$Restart)
2. **Run all cells** (Cell$\rightarrow$Run All)
3. **Save** (File$\rightarrow$Save and Checkpoint)

<div class="alert alert-danger">After you have completed these three steps, ensure that the following cell has printed "No errors". If it has <b>not</b> printed "No errors", then your code has a bug in it and has thrown an error! Make sure you fix this error before turning in your problem set.</div>

In [ ]:
print("No errors!")